In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import nltk

In [25]:
pd.options.display.max_seq_items = 2000

In [46]:
datasheet = pd.read_excel('CogData_FU.xlsx',header=None)

In [47]:
len(datasheet)

577

In [48]:
datasheet.head()

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,Record ID,Event Name,Write a short sketch about a memory from your ...,How does technology and social media impact th...,Date of Neurobehavioral Status Exam,Notes,NIH Toolbox ID,NIH Toolbox Password,Crystallized Composite fully adjusted,Picture Vocab fully adjusted,...,Date Conducted:,CogState: Detection,CogState: Identification,CogState: One Card Learning,CogState: One Back Speed,Cog State: One Back Accuracy,Ravens Progressive Matrices,Logical Memory Part B,East Boston Immediate Recall,Complete?
1,10,Baseline,NaN,NaN,2015-02-23 00:00:00,"RAVLT recognition, Odor ID and Trails B not ad...",85176,NaN,83.32,73.6571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
2,10,FU Visit 2,NaN,NaN,2015-09-28 00:00:00,Trails B incomplete,65967,NaN,129.69,104.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
3,13,Baseline,NaN,NaN,2015-02-24 00:00:00,Odor ID not administered,15497,NaN,140.09,122.101,...,2015-04-01 00:00:00,106,109,110,101,106,NaN,NaN,NaN,Complete
4,13,FU Visit 2,NaN,NaN,2016-02-12 00:00:00,NaN,12037,awk,143.18,131.251,...,2016-02-12 00:00:00,105,107,108,103,106,NaN,NaN,NaN,Complete


In [88]:
#ignore first (header) column
#indices of dataset where 
memory_ix = datasheet.iloc[1:][datasheet.iloc[1:,2].notnull()].index

In [101]:
memory_id = list(datasheet.iloc[memory_ix][0])

In [100]:
len(memory_ix)

46

In [98]:
tech_ix = datasheet.iloc[1:][datasheet.iloc[1:,3].notnull()].index

In [99]:
tech_ix

Int64Index([  6,  29,  51,  56,  78,  89, 100, 107, 118, 154, 184, 202, 217,
            226, 234, 242, 266, 277, 280, 286, 297, 303, 324, 337, 347, 352,
            359, 375, 387, 398, 411, 423, 433, 438, 445, 448, 460, 464, 470,
            474, 477, 496, 499, 502, 506, 509, 519, 522, 537, 540],
           dtype='int64')

In [102]:
tech_id = list(datasheet.iloc[tech_ix][0])

In [108]:
#row indices of tech-respondents, mem-negligent
[tix for tix in tech_ix if tix not in memory_ix]

[234, 280, 337, 519]

In [106]:
#record ids of tech-respondents who are mem-negligent
[tid for tid in tech_id if tid not in memory_id]

[271, 320, 403, 616]

In [107]:
#mem respondents who are tech-negligent
[m for m in memory_id if m not in tech_id]

[]

In [114]:
tech_exclusive_id = [tid for tid in tech_id if tid not in memory_id]

In [ ]:
#Which record IDs responded

In [77]:
datasheet.iloc[tech_exclusive]

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
234,271,Baseline,NaN,I'm writing correspondence via computer most o...,2015-07-14 00:00:00,No RAVLT rec.,6578,NaN,91.1,96.3082,...,2015-10-02 00:00:00,108,105,105,99,75,NaN,NaN,NaN,Complete
280,320,Baseline,NaN,Does not affect my life or family.,2015-11-03 00:00:00,NaN,31043,qxy,148.23,132.234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
337,403,Baseline,NaN,Technology has interrupted my personal life th...,2015-12-01 00:00:00,NaN,60023,ytv,141.23,129.919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
519,616,Baseline,NaN,I feel that a lot of times technology and soci...,2017-03-30 00:00:00,NaN,34946,snz,132.84,130.988,...,2017-03-30 00:00:00,100,95,107,90,116,NaN,NaN,NaN,Complete


In [117]:
#all rows for tech_respondents
tech_all_ix = [ix for ix in datasheet.index if datasheet.iloc[ix,0] in tech_id]

In [118]:
datasheet.iloc[tech_all_ix]

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
6,14,Baseline,When I was a little child I had the same exper...,I stay in touch with family and friends throug...,2015-02-26 00:00:00,NaN,35926,qjh,123.54,126.545,...,2015-03-17 00:00:00,102,99,103,93,116,NaN,NaN,NaN,Complete
7,14,FU Visit 2,NaN,NaN,2015-07-14 00:00:00,NaN,82858,NaN,126.32,124.077,...,2015-07-02 00:00:00,108,99,95,91,107,NaN,NaN,NaN,Complete
8,14,FU Visit 3,NaN,NaN,2016-03-07 00:00:00,NaN,48846,evf,132.09,132.171,...,2016-03-07 00:00:00,102,98,110,92,101,NaN,NaN,NaN,Complete
9,14,FU Visit 4,NaN,NaN,2016-08-25 00:00:00,NaN,94255,yua,126.02,132.98,...,2016-08-25 00:00:00,105,94,104,86,103,NaN,NaN,NaN,Complete
10,14,FU Visit 5,NaN,NaN,2017-02-17 00:00:00,NaN,45760,mur,134.39,134.534,...,2017-02-17 00:00:00,104,98,106,93,101,NaN,NaN,NaN,Complete
11,14,FU Visit 6,NaN,NaN,2017-09-15 00:00:00,NaN,84695,hfa,120.9,130.529,...,2017-09-15 00:00:00,102,98,105,93,103,NaN,NaN,NaN,Complete
12,14,FU Visit 7,NaN,NaN,2018-04-16 00:00:00,NaN,12836,wtc,132.08,135.857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
29,26,Baseline,"When I was very young, living in the Astoria n...",I'm a journalist who has covered the rise of t...,2015-02-03 00:00:00,RAVLT rec not administered,36080,NaN,148.32,145.587,...,2015-02-03 00:00:00,113,112,100,100,103,NaN,NaN,NaN,Complete
30,26,FU Visit 2,NaN,NaN,2015-07-28 00:00:00,"No picture vocab administered and thus, no cry...",44535,zsk,NaN,NaN,...,2015-07-29 00:00:00,NaN,107,79,96,60,NaN,NaN,NaN,Complete
31,26,FU Visit 3,NaN,NaN,2016-01-28 00:00:00,NaN,98969,zsk,153.68,145.569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete


In [119]:
mem_all_ix = [ix for ix in datasheet.index if datasheet.iloc[ix,0] in memory_id]

In [120]:
datasheet.iloc[mem_all_ix]

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
6,14,Baseline,When I was a little child I had the same exper...,I stay in touch with family and friends throug...,2015-02-26 00:00:00,NaN,35926,qjh,123.54,126.545,...,2015-03-17 00:00:00,102,99,103,93,116,NaN,NaN,NaN,Complete
7,14,FU Visit 2,NaN,NaN,2015-07-14 00:00:00,NaN,82858,NaN,126.32,124.077,...,2015-07-02 00:00:00,108,99,95,91,107,NaN,NaN,NaN,Complete
8,14,FU Visit 3,NaN,NaN,2016-03-07 00:00:00,NaN,48846,evf,132.09,132.171,...,2016-03-07 00:00:00,102,98,110,92,101,NaN,NaN,NaN,Complete
9,14,FU Visit 4,NaN,NaN,2016-08-25 00:00:00,NaN,94255,yua,126.02,132.98,...,2016-08-25 00:00:00,105,94,104,86,103,NaN,NaN,NaN,Complete
10,14,FU Visit 5,NaN,NaN,2017-02-17 00:00:00,NaN,45760,mur,134.39,134.534,...,2017-02-17 00:00:00,104,98,106,93,101,NaN,NaN,NaN,Complete
11,14,FU Visit 6,NaN,NaN,2017-09-15 00:00:00,NaN,84695,hfa,120.9,130.529,...,2017-09-15 00:00:00,102,98,105,93,103,NaN,NaN,NaN,Complete
12,14,FU Visit 7,NaN,NaN,2018-04-16 00:00:00,NaN,12836,wtc,132.08,135.857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
29,26,Baseline,"When I was very young, living in the Astoria n...",I'm a journalist who has covered the rise of t...,2015-02-03 00:00:00,RAVLT rec not administered,36080,NaN,148.32,145.587,...,2015-02-03 00:00:00,113,112,100,100,103,NaN,NaN,NaN,Complete
30,26,FU Visit 2,NaN,NaN,2015-07-28 00:00:00,"No picture vocab administered and thus, no cry...",44535,zsk,NaN,NaN,...,2015-07-29 00:00:00,NaN,107,79,96,60,NaN,NaN,NaN,Complete
31,26,FU Visit 3,NaN,NaN,2016-01-28 00:00:00,NaN,98969,zsk,153.68,145.569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete


In [78]:
#(index,col_name)
[(i,score) for (i,score) in enumerate(datasheet.iloc[0])]

[(0, 'Record ID'),
 (1, 'Event Name'),
 (2,
  'Write a short sketch about a memory from your childhood and why it is memorable or important to you. Feel free to discuss an event with family or friends, a place you traveled, or a significant time in your life.  Please limit your response to no more than 1-2 paragraphs.  '),
 (3,
  'How does technology and social media impact the daily lives of you and your family?Please limit your response to no more than 1-2 paragraphs.  '),
 (4, 'Date of Neurobehavioral Status Exam'),
 (5, 'Notes'),
 (6, 'NIH Toolbox ID'),
 (7, 'NIH Toolbox Password'),
 (8, 'Crystallized Composite fully adjusted'),
 (9, 'Picture Vocab fully adjusted'),
 (10, 'Picture Vocab age adjusted'),
 (11, 'Picture Vocab computed'),
 (12, 'Oral Reading Recognition fully adjusted'),
 (13, 'Oral Reading Recognition age adjusted'),
 (14, 'Oral Reading Recognition computed'),
 (15, 'RAVLT Trial1'),
 (16, 'RAVLT Trial2'),
 (17, 'RAVLT Trial3'),
 (18, 'RAVLT Delayed Recall'),
 (19, 'RA

In [83]:
len(datasheet.iloc[tech_responses][74])

50

In [121]:
datasheet.iloc[tech_ix][74]

6       30
29      30
51      30
56      30
78      29
89      30
100     29
107    NaN
118     29
154     30
184     29
202     30
217     30
226     30
234     28
242     30
266     30
277     30
280     30
286     30
297     30
303     28
324     30
337     30
347     29
352     29
359     30
375     28
387     30
398     29
411     30
423     30
433     30
438     30
445     30
448     29
460     29
464     30
470     30
474     30
477     29
496     29
499     30
502     29
506     29
509     30
519     30
522     30
537     29
540     27
Name: 74, dtype: object

In [122]:
#MMSE scores
datasheet.iloc[tech_responses][74].describe()

count     49
unique     4
top       30
freq      31
Name: 74, dtype: int64

In [124]:
datasheet.iloc[tech_all_ix][74].describe()

count     220
unique      6
top        30
freq      152
Name: 74, dtype: int64

In [129]:
np.argmin(datasheet.iloc[tech_all_ix][74])

390

In [145]:
#The maximum MMSE score is 30 points. A score of 20 to 24 suggests mild dementia, 13 to 20 suggests moderate dementia, and less than 12 indicates severe dementia.
#participants with MCI...one ---
datasheet.iloc[tech_all_ix][74][datasheet.iloc[tech_all_ix][74] < 25]

390    23
Name: 74, dtype: object

In [38]:
datasheet.iloc[tech_responses]

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,Record ID,Event Name,Write a short sketch about a memory from your ...,How does technology and social media impact th...,Date of Neurobehavioral Status Exam,Notes,NIH Toolbox ID,NIH Toolbox Password,Crystallized Composite fully adjusted,Picture Vocab fully adjusted,...,Date Conducted:,CogState: Detection,CogState: Identification,CogState: One Card Learning,CogState: One Back Speed,Cog State: One Back Accuracy,Ravens Progressive Matrices,Logical Memory Part B,East Boston Immediate Recall,Complete?
6,14,Baseline,When I was a little child I had the same exper...,I stay in touch with family and friends throug...,2015-02-26 00:00:00,NaN,35926,qjh,123.54,126.545,...,2015-03-17 00:00:00,102,99,103,93,116,NaN,NaN,NaN,Complete
29,26,Baseline,"When I was very young, living in the Astoria n...",I'm a journalist who has covered the rise of t...,2015-02-03 00:00:00,RAVLT rec not administered,36080,NaN,148.32,145.587,...,2015-02-03 00:00:00,113,112,100,100,103,NaN,NaN,NaN,Complete
51,64,Baseline,I went to Paris when I was 16. I was staying ...,My husband and I are on our computers all day ...,2015-05-08 00:00:00,No Rey recognition,61681,NaN,130.68,134.57,...,2015-06-16 00:00:00,NaN,NaN,80,96,61,NaN,NaN,NaN,Complete
56,67,Baseline,My husband and I had always wanted to go to Pa...,"I'm on the computer all the time, writing or r...",2015-03-17 00:00:00,No RAVLT recognition and Odor ID,56478,NaN,130.99,129.869,...,2015-03-25 00:00:00,96,105,109,99,103,NaN,NaN,NaN,Complete
78,101,Baseline,I was born in Hungary. A significant memory is...,My use of social media is minimal. I only use ...,2015-03-06 00:00:00,No RAVLT rec and Odor ID,52303,NaN,123.55,117.09,...,2015-04-22 00:00:00,NaN,113,72,NaN,NaN,NaN,NaN,NaN,Complete
89,110,Baseline,When I was 16 years old I traveled to Europe b...,I check that on what my children are doing and...,2016-03-04 00:00:00,NaN,78039,vab,137.14,133,...,2016-03-04 00:00:00,106,105,112,102,103,NaN,NaN,NaN,Complete
100,135,Baseline,I have great memories of growing up in huge ho...,Technology has helped me connect to family and...,2015-04-20 00:00:00,No RAVLT rec.,59106,NaN,105.68,106.366,...,2015-04-28 00:00:00,92,94,105,100,116,NaN,NaN,NaN,Complete
107,139,Baseline,"When I was 8 or 9 years old, I was playing on ...",I use a laptop and phone constantly for work a...,2015-12-23 00:00:00,No record of paper-based tests found. RAVLT no...,11825,NaN,113.41,108.199,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
118,145,Baseline,One of my fondest childhood memories is how mu...,I do not use any social media sites at this ti...,2015-05-05 00:00:00,No RAVLT rec.,71700,NaN,98.08,94.6693,...,2015-05-06 00:00:00,108,110,96,111,116,NaN,NaN,NaN,Complete
